### This notebook will mainly be used for the Capstone Project

    #### Import required Libraries

In [18]:
import pandas as pd
import os
os.system('pip install wikipedia')
os.system('pip install geocoder')
os.system('pip install geopy')
os.system('pip install folium')
import wikipedia as wp
from geopy.geocoders import Nominatim
import folium



#### Call the wikipedia page function and pass the "list of postal codes of Canada" page, read the html and create the df

In [19]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.columns = ['Postalcode', 'Borough', 'Neighborhood']
df = df.iloc[1:]
#df.to_csv('C:\\Users\RAGHUGOPALKRISHNAN\\Desktop\\Course9.csv')
#print(df)


#### Use groupby to group on the Postalcode column and aggregate on the Neighborhood column. Drop rows which have 'not assigned' in the Borough column and assign Borough value if the Neighborhood has value as 'Not assigned'

In [20]:
foo = lambda a: " , ".join(a) 
df = df.groupby('Postalcode').agg({'Borough': 'first', 'Neighborhood': foo}).reset_index()
   
    
df = df.drop(df[(df.Borough == 'Not assigned')].index)
for index, row in df.iterrows():
    if row['Neighborhood'] == 'Not assigned':
        print(row['Borough'])
        row['Neighborhood'] = row['Borough']



Queen's Park


#### Due to unreliability of the geocoder package I am using the geospatial csv file to get the latitude and longtitude for the Postal Codes

In [21]:
geospatial = pd.read_csv('https://cocl.us/Geospatial_data')
#geospatial
geospatial.columns = ['Postalcode', 'Latitude', 'Longitude']
#geospatial
df_merge = pd.merge(df, geospatial, on='Postalcode')
df_merge

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge , Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park , Ionview , Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea , Golden Mile , Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest , Cliffside , Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848


#### Get Latitude and Longitude of Downtown Toronto

In [22]:
address = 'Downtown Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.3808116451341.


#### Create a map of Downtown Toronto with neighborhoods superimposed on top

In [23]:
from IPython.display import display
Downtown_Toronto = df_merge[df_merge['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.head()
Latitude = 43.654027
Longitude = -79.3802003
map_Downtown_Toronto = folium.Map(location=[Latitude, Longitude], zoom_start=4)
for lat, lng, label in zip(Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Downtown_Toronto)
    
# map_Downtown_Toronto.save('map_Downtown_Toronto.html')
# f = codecs.open("map_Downtown_Toronto.html", 'r', 'utf-8')
# f
display(map_Downtown_Toronto)

#### Define Foursquare credentials and version

In [24]:
CLIENT_ID = 'BLLCQDJ141BQCBPOYYMFN5VUIFH5ZR4NDTDFRP52QHQM2CPK' # your Foursquare ID
CLIENT_SECRET = 'G20BXB1EQAM0ZSJF35AOMV12LDF11ODIOH0XRVIATC0OLUZN' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 30

#### Create a function to explore neighborhoods 

In [25]:
import requests
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Explore each neighborhood using the above function and create a new Df called Downtown_Toronto_venues

In [26]:
Downtown_Toronto_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'],
                                   latitudes=Downtown_Toronto['Latitude'],
                                   longitudes=Downtown_Toronto['Longitude']
                                  )
Downtown_Toronto_venues

Rosedale
Cabbagetown , St. James Town
Church and Wellesley
Harbourfront , Regent Park
Ryerson , Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide , King , Richmond
Harbourfront East , Toronto Islands , Union Station
Design Exchange , Toronto Dominion Centre
Commerce Court , Victoria Hotel
Harbord , University of Toronto
Chinatown , Grange Park , Kensington Market
CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place , Underground city
Christie


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"Cabbagetown , St. James Town",43.667967,-79.367675,Cranberries,43.667843,-79.369407,Diner
5,"Cabbagetown , St. James Town",43.667967,-79.367675,Butter Chicken Factory,43.667072,-79.369184,Indian Restaurant
6,"Cabbagetown , St. James Town",43.667967,-79.367675,F'Amelia,43.667536,-79.368613,Italian Restaurant
7,"Cabbagetown , St. James Town",43.667967,-79.367675,Kingyo Toronto,43.665895,-79.368415,Japanese Restaurant
8,"Cabbagetown , St. James Town",43.667967,-79.367675,Murgatroid,43.667381,-79.369311,Restaurant
9,"Cabbagetown , St. James Town",43.667967,-79.367675,Cabbagetown Brew,43.666923,-79.369289,Café
